# LipConstEstimator: IPython Demo
This notebook is a brief demo for the package ECLispE.

In [1]:
import os
# Prevent OpenMP/MKL fights between torch/numpy/scs on Windows
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch.nn as nn
import numpy as np
from python.LipConstEstimator import LipConstEstimator
import os
import torch

## Create Estimator by Torch Model

In [2]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(20, 5)
        self.act2 = nn.Sigmoid()
    
    def forward(self, x):
        x = self.act1(self.fc1(x))
        return self.act2(self.fc2(x))

model = SimpleNet()
est = LipConstEstimator(model=model)
est.model_review()


MODEL INFO
Layer #0: input size = 10, output size = 20, activation = ReLU.
Layer #1: input size = 20, output size = 5, activation = Sigmoid.
REMARK: ONLY FULLY CONNECT NEURAL NETS ARE APPLICABLE IN THIS ESTIMATOR.


In [3]:
lip_trivial = est.estimate(method='trivial')
lip_fast = est.estimate(method='ECLipsE_Fast')
lip_acc = est.estimate(method='ECLipsE')
print(f'Trivial Lip Const = {lip_trivial}')
print(f'EClipsE Fast Lip Const = {lip_fast}')
print(f'EClipsE Lip Const = {lip_acc}')
print(f'Ratio Fast = {lip_fast / lip_trivial}')
print(f'Ratio Acc = {lip_acc / lip_trivial}')

C:\Users\22384\anaconda3\Lib\site-packages\scs\__init__.py:96: UserWarning: Converting A to a CSC (compressed sparse column) matrix; may take a while.
  warn(


Trivial Lip Const = 3.452872006114886
EClipsE Fast Lip Const = 0.8026592986092945
EClipsE Lip Const = 0.7226857738027781
Ratio Fast = 0.23246135309615296
Ratio Acc = 0.2092999023777693


## Create Estimator by Given Weights

In [4]:
weights_npz = np.load('sampleweights' + os.sep + 'npz' + os.sep + 'lyr' + str(2) + 'n' + str(80) + 'test' + str(1) + '.npz')
weights = []
for i in range(1,2+1):
    weights.append(torch.tensor(weights_npz['w'+str(i)]))
est = LipConstEstimator(weights=weights)
lip_trivial = est.estimate(method='trivial')
lip_fast = est.estimate(method='ECLipsE_Fast')
print(f'Trivial Lip Const = {lip_trivial}')
print(f'EclipsE Fast Lip Const = {lip_fast}')
print(f'Ratio = {lip_fast / lip_trivial}')

Trivial Lip Const = 0.9676370477588296
EclipsE Fast Lip Const = 0.8090351972788796
Ratio = 0.8360936563484294


## Create Estimator by Nothing (Randomly Generate a FNN)

In [5]:
est = LipConstEstimator()
est.generate_random_weights([10,40,3])
lip_trivial = est.estimate(method='trivial')
lip_fast = est.estimate(method='ECLipsE_Fast')
print(f'Trivial Lip Const = {lip_trivial}')
print(f'EclipsE Fast Lip Const = {lip_fast}')
print(f'Ratio = {lip_fast / lip_trivial}')

Trivial Lip Const = 74.038904541446
EclipsE Fast Lip Const = 57.879768083601235
Ratio = 0.7817480342540847
